In [1]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download("stopwords')
from nltk.stem import WordNetLemmatizer
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from nltk.corpus import stopwords
import re 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import time

In [2]:
data= sc.textFile("fake_true.csv")



In [3]:
data.count()

6342

In [4]:
data = data.filter(lambda x: "label" not in x)

In [6]:
data.take(2)

['smell hillary fear;daniel greenfield shillman journalism fellow freedom center new york writer focusing radical islam final stretch election hillary rodham clinton gone war fbi word unprecedented thrown around often election ought retired still unprecedented nominee major political party go war fbi exactly hillary people done coma patients waking watching hour cnn hospital beds would assume fbi director james comey hillary opponent election fbi attack everyone obama cnn hillary people circulated letter attacking comey currently media hit pieces lambasting targeting trump surprising clintons allies start running attack ads fbi fbi leadership warned entire left wing establishment form lynch mob continue going hillary fbi credibility attacked media democrats preemptively head results investigation clinton foundation hillary clinton covert struggle fbi agents obama doj people gone explosively public new york times compared comey j edgar hoover bizarre headline james comey role recalls ho

In [7]:
def word_tokenize1(x):
    x =re.sub('[^a-zA-Z]' ,' ', x)
    x = x.lower()
    return nltk.word_tokenize(x)

   

def lemma(x):
    lemmatizer = WordNetLemmatizer()

    z=[lemmatizer.lemmatize(word) for word in x  if not word in set (stopwords.words('english'))]
    z = [lemmatizer.lemmatize(word,pos='v') for word in z]
  
    z = ' '.join(z)
    return z

In [28]:
def preprocessing(inputs):
    inputss = inputs.split(";")
    title=word_tokenize1(inputss[0])
    title=lemma(title)
    text=word_tokenize1(inputss[1])
    text=lemma(text)
   
    if inputss[2]=="0":
        LABEL=0.0
    else:
        LABEL=1.0
    return [LABEL,text,title]

In [9]:

data= data.map(preprocessing)



In [10]:
#pretime

In [11]:
data.take(2)

[[0.0,
  'daniel greenfield shillman journalism fellow freedom center new york writer focus radical islam final stretch election hillary rodham clinton go war fbi word unprecedented throw around often election ought retire still unprecedented nominee major political party go war fbi exactly hillary people do coma patient wake watch hour cnn hospital bed would assume fbi director jam comey hillary opponent election fbi attack everyone obama cnn hillary people circulate letter attack comey currently medium hit piece lambast target trump surprise clinton ally start run attack ad fbi fbi leadership warn entire leave wing establishment form lynch mob continue go hillary fbi credibility attack medium democrat preemptively head result investigation clinton foundation hillary clinton covert struggle fbi agent obama doj people go explosively public new york time compare comey j edgar hoover bizarre headline jam comey role recall hoover fbi fairly practically admit front spout nonsense boston gl

In [12]:
df= sqlContext.createDataFrame(data,["label","text","title"])

In [13]:
df=df.dropna()
df.show()

+-----+--------------------+--------------------+
|label|                text|               title|
+-----+--------------------+--------------------+
|  0.0|daniel greenfield...|  smell hillary fear|
|  0.0|google pinterest ...|watch exact momen...|
|  1.0|u secretary state...|kerry go paris ge...|
|  0.0|kaydee king kayde...|bernie supporter ...|
|  1.0|primary day new y...|battle new york p...|
|  0.0|immigrant grandpa...|          tehran usa|
|  0.0|share baylee luci...|girl horrify watc...|
|  1.0|czech stockbroker...|britain schindler dy|
|  1.0|hillary clinton d...|fact check trump ...|
|  1.0|iranian negotiato...|iran reportedly m...|
|  1.0|donald trump orga...|donald trump shoc...|
|  0.0|click learn alexa...|strong solar stor...|
|  0.0|october pretty fa...|way america prepa...|
|  1.0|kill obama admini...|trump take cruz l...|
|  1.0|woman move high o...|woman lead differ...|
|  0.0|shock michele oba...|shock michele oba...|
|  0.0|hillary clinton b...|hillary clinton h...|


In [14]:

(trainingData,testData)= df.randomSplit([0.8,0.2])


In [16]:
#testData.take(2)

#t2=time.time()
#trainingData.take(2)
#taketime=(time.time())-t2

#taketime

In [15]:
#take care it is the Pipeline not pipeline
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF,Tokenizer
from pyspark.ml.feature import IDF

In [17]:
#Tokenize each message (Split it into words)
tokenizer = Tokenizer(inputCol="text",outputCol="words")

#Build Text frequency 
hashingTF= HashingTF(inputCol=tokenizer.getOutputCol(),outputCol="tempfeatures")

#Build Inverted text frequency
idf = IDF(inputCol=hashingTF.getOutputCol(),outputCol="features")

#Building the Model

from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()
#lrModel = lr.fit(train)

In [18]:
#One flow in Pipeline
# Initiate a pipeline. I could've done by individually tokenizer.fit(), 
#hashing_tf.fit(), idf.fit() etc.. But pipeline does this in one shot
pipeline = Pipeline(stages = [tokenizer, hashingTF, idf, lr])

In [19]:
# Fit the training data to the classifier to create the Naive Bayes Model
t3=time.time()
lrModue = pipeline.fit(trainingData)
modeltime=(time.time())-t3


In [22]:
modeltime

391.46261858940125

In [23]:
t4=time.time()
predictions= lrModue.transform(testData)

evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",
                                              labelCol="label",
                                              metricName="precision")

evaluator.evaluate(predictions)
evals=(time.time())-t4

0.9103214890016921

In [27]:



# Form the confusion matrix of the data
predictions.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  524|
|  0.0|       0.0|  552|
|  0.0|       1.0|   56|
|  1.0|       0.0|   50|
+-----+----------+-----+

